In [225]:
# read the data
#imports"
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Draw

# Create a Path object for the current directory
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

# Creating a Path object for an example file that does not yet exist
example_file_path = current_directory / "database.csv"

# Reading the contents of the file to test if we have well acess to our data
if example_file_path.exists():
    with example_file_path.open("r") as file:
        first_line = file.readline()
        print(first_line)
else:
    print("The file does not exist.")

Current Directory: C:\Users\33783\Documents\EPFL\Etudes\BA4\Amazing-Project
Reactant 1;Reactant 2;Product 1;Product 2;Product 3;yield;reaction_smarts



In [226]:
# Create the Data Frame
from rdkit import Chem
from rdkit.Chem import PandasTools
import pandas as pd
from pathlib import Path
import os

dataFrame= pd.read_csv("database.csv")
print (dataFrame)

     Reactant 1;Reactant 2;Product 1;Product 2;Product 3;yield;reaction_smarts
0     [Br-:2];[Br:1][C:5]([C:4]([H:3])([C:31]#[N:32]...                       
1     [Br-:2];[Br:1][C:5]([C:4]([H:3])([C:31]#[N:32]...                       
2     [Br-:2];[Br:1][C:5]([C:4]([H:3])([H:31])[C:41]...                       
3     [Br-:2];[Br:1][C:5]([C:4]([H:3])([H:31])[H:41]...                       
4     [Br-:2];[Br:1][C:5]([C:4]([H:3])([H:31])[H:41]...                       
...                                                 ...                       
3620  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...                       
3621  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...                       
3622  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...                       
3623  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...                       
3624  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...                       

[3625 rows x 1 columns]


In [227]:
# fonction to remove atome mapping numbers from smiles
import re

def remove_atom_mapping(smiles):
    """
    Remove atom mapping numbers from a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation with atom mapping numbers.
    Returns:
    - smiles_without_mapping (str): The SMILES notation without atom mapping numbers.
    """
    # Define a regular expression pattern to match atom mapping numbers
    pattern = r':\d+'
    smiles_without_mapping = re.sub(pattern, '', smiles)
    return smiles_without_mapping

In [242]:
# give smiles

import re

def Smiles(smiles):
    """
    Remove atom mapping numbers, brackets, 'H' characters, and empty parentheses from a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation with atom mapping numbers, brackets, 'H' characters, and empty parentheses.
    Returns:
    - smiles_cleaned (str): The cleaned SMILES notation.
    """
    # Define a regular expression pattern to match atom mapping numbers, brackets, 'H' characters, and empty parentheses
    pattern = r'[:\[\]\ H@\/]'
    # Use re.sub() to remove the specified characters
    smiles_cleaned = re.sub(pattern, '', smiles)
    # Remove empty parentheses only
    smiles_cleaned = re.sub(r'\(\)', '', smiles_cleaned)
    return smiles_cleaned

def remove_forward_slash(smiles):
    """
    Remove forward slash '(\)' from a string.
    Args:
    - smiles (str): The input string.
    Returns:
    - cleaned_str (str): The string with forward slash removed.
    """
    return smiles.replace(r'(\)', '')


def convert_X(smiles):
    """
    Convert "X-" elements where X can be "Br", "Cl", or "F" into "[X-]" in a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation.
    Returns:
    - cleaned_smiles (str): The cleaned SMILES notation.
    """
    # Define the elements you want to convert
    elements = ['Br', 'Cl', 'F']
    # Iterate over each element
    for element in elements:
        # Replace "X-" with "[X-]"
        smiles = smiles.replace(element + '-', '[' + element + '-]')
    return smiles

def transform_anions(smiles):
    """
    Transform the input string by replacing '(N+(O-)O)' with 'NO'.
    Args:
    - input_string (str): The input string to be transformed.
    Returns:
    - transformed_string (str): The transformed string.
    """
    transformed_string = re.sub(r'N\+\(O-\)O', 'NO', smiles)
    return transformed_string


In [243]:
# remove mapping from the Data Frame
for column in dataFrame.columns:
    # Apply the remove_atom_mapping function to each column
    dataFrame[column] = dataFrame[column].apply(remove_atom_mapping)
    dataFrame[column] = dataFrame[column].apply(remove_forward_slash)
    dataFrame[column] = dataFrame[column].apply(Smiles)
    dataFrame[column] = dataFrame[column].apply(convert_X)
    dataFrame[column] = dataFrame[column].apply(transform_anions)
print(dataFrame)

     Reactant 1;Reactant 2;Product 1;Product 2;Product 3;yield;reaction_smarts
0     [Br-];BrC(C(C#N)C#N);[Br-];BrC(C(C#N)C#N);;28....                       
1     [Br-];BrC(C(C#N))(C)C;[Br-];BrC(C(C#N))(C)C;;3...                       
2     [Br-];BrC(CC#N)(C)C;[Br-];BrC(CC#N)(C)C;;37.64...                       
3     [Br-];BrC(C)(C);[Br-];BrC(C)(C);;24.2066422218...                       
4     [Br-];BrC(C)C;[Br-];BrC(C)C;;18.45075294694804...                       
...                                                 ...                       
3620  FC(C(N)NO)N;-;[F-];C(C(N)NO)N;;26.844888319262...                       
3621  FC(C(N)NO)(NO)C;-;C(=C(NO)C)(\N)NO;[F-];;2.283...                       
3622  FC(C(N)NO)(NO);-;[F-];C(C(N)NO)(NO);;18.142052...                       
3623  FC(C(N)NO)(N)C#N;-;[F-];C(C(N)NO)(N)C#N;;18.12...                       
3624  FC(C(N)NO)(N);-;[F-];C(C(N)NO)(N);;30.15313442...                       

[3625 rows x 1 columns]


In [244]:
# Split the column values at each ";" and expand them into separate columns
split_dataFrame = dataFrame['Reactant 1;Reactant 2;Product 1;Product 2;Product 3;yield;reaction_smarts'].str.split(';', expand=True)

# Rename the columns
split_dataFrame.columns = ['Reactant 1', 'Reactant 2', 'Product 1', 'Product 2', 'Product 3', 'yield', "reaction_smarts"]

# Display the resulting DataFrame
print(split_dataFrame)

            Reactant 1       Reactant 2         Product 1        Product 2  \
0                [Br-]   BrC(C(C#N)C#N)             [Br-]   BrC(C(C#N)C#N)   
1                [Br-]  BrC(C(C#N))(C)C             [Br-]  BrC(C(C#N))(C)C   
2                [Br-]    BrC(CC#N)(C)C             [Br-]    BrC(CC#N)(C)C   
3                [Br-]        BrC(C)(C)             [Br-]        BrC(C)(C)   
4                [Br-]          BrC(C)C             [Br-]          BrC(C)C   
...                ...              ...               ...              ...   
3620       FC(C(N)NO)N                -              [F-]       C(C(N)NO)N   
3621   FC(C(N)NO)(NO)C                -  C(=C(NO)C)(\N)NO             [F-]   
3622    FC(C(N)NO)(NO)                -              [F-]    C(C(N)NO)(NO)   
3623  FC(C(N)NO)(N)C#N                -              [F-]  C(C(N)NO)(N)C#N   
3624     FC(C(N)NO)(N)                -              [F-]     C(C(N)NO)(N)   

     Product 3               yield  \
0                28.81200

In [245]:
# Assuming your DataFrame is named dataFrame
# Access the third column and get its first element
element = split_dataFrame.iloc[0, 2]

# Print the element and its type
print("First element of the third column:", element)
print("Type of the element:", type(element))

First element of the third column: [Br-]
Type of the element: <class 'str'>


In [232]:
# Prompt the user to enter the molecule that he wants to form
print ("you can enter the name of the molecule or its SMILES, the racemic configuration is available. Only halogen molecules are taken in account in the data base")
string_input_mol = input("Enter the molecule that you want to form: ")

you can enter the name of the molecule or its SMILES, the racemic configuration is available. Only halogen molecules are taken in account in the data base


Enter the molecule that you want to form:  bromide


In [246]:
# function to determine if the user enters the SMILES notation or the usual name of the molecule
from rdkit import Chem

def is_smiles(smiles):
    """
    Check if a string represents a valid SMILES notation.
    Args:
    - smiles (str): The string to check.
    Returns:
    - is_valid (bool): True if the string is a valid SMILES notation, False otherwise.
    """
    if Chem.MolFromSmiles(smiles)== None:
        return False
    else:
        return True

# Check if the entered string is a valid SMILES notation
if is_smiles(string_input_mol):
    print(f"'{string_input_mol}' is a valid SMILES notation.")
else:
    print(f"'{string_input_mol}' is not a valid SMILES notation.")

'[Br-]' is a valid SMILES notation.


In [247]:
!pip install pubchempy

In [248]:
# function to convert name into SMILES

import pubchempy as pcp

def name_to_smiles(molecule_name):
    """
    Convert a molecule name to a SMILES notation using PubChemPy's PubChem database.
    Args:
    - molecule_name (str): The name of the molecule.
    Returns:
    - smiles (str): The SMILES notation of the molecule, or None if retrieval fails.
    """
    try:
        compound = pcp.get_compounds(molecule_name, 'name')
        if compound:
            return compound[0].canonical_smiles
        else:
            print("Error: Unable to retrieve molecule information. Please try with the SMILE of the molecule")
            return None
    except:
        print("Error: Unable to retrieve molecule information. Please try again")
        return None

In [249]:
# Call the function to convert the molecule name to SMILES notation
if is_smiles(string_input_mol)== False:
    string_input_mol = name_to_smiles(string_input_mol)
    if string_input_mol:
        print(f"SMILES notation for {string_input_mol}: {string_input_mol}")
else:
    print (string_input_mol)

[Br-]


In [250]:
# function to find the required molecule in the data Frame

def compare_molecule_with_data(element):
    return element == string_input_mol

# Iterate over the last three columns of the DataFrame
for column_name in split_dataFrame.columns[-4:-1]:
    column = split_dataFrame[column_name]
    # Check if any element in the column matches the input molecule
    if any(column.apply(compare_molecule_with_data)):
        print("Input molecule found in the DataFrame:")
        print(split_dataFrame[column.apply(compare_molecule_with_data)])
        print("Row number:", index)
        break
else:
    print("The product is not in the database")

The product is not in the database


In [251]:
# Function that returns the row of the elements formation reaction
def compare_molecule_with_data(element):
    return element == string_input_mol

# Initialize a variable to store the row number
found_row_number = None
rows=[]
# Iterate over the last three columns of the DataFrame
for column_name in split_dataFrame.columns[-5:-1]:
    column = split_dataFrame[column_name]
    # Check if any element in the column matches the input molecule
    for index, value in column.items():
        if compare_molecule_with_data(value):
            # Store the row number where the molecule is found
            found_row_number = index
            rows.append(found_row_number)
    else:
        continue  # Continue to the next column if molecule not found in current column

# Check if the molecule was found and return the row number
if found_row_number is not None:
   print(rows)
else:
    print("The product is not in the database")


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508,

In [252]:
# print the yield
# Convert the 'Yield' column to float
split_dataFrame['yield'] = split_dataFrame['yield'].astype(float)

# Assuming 'rows_to_compare' is a list containing the row numbers to compare

# Subset the DataFrame to include only the specified rows
subset_df = split_dataFrame.iloc[rows]
print(subset_df)

# Find the index of the row with the highest yield
max_yield_row_index = subset_df['yield'].idxmax()

# Retrieve the row with the highest yield
max_yield_row = split_dataFrame.loc[max_yield_row_index]

print("Row with the highest yield:")
print(max_yield_row)


            Reactant 1       Reactant 2 Product 1        Product 2 Product 3  \
0                [Br-]   BrC(C(C#N)C#N)     [Br-]   BrC(C(C#N)C#N)             
1                [Br-]  BrC(C(C#N))(C)C     [Br-]  BrC(C(C#N))(C)C             
2                [Br-]    BrC(CC#N)(C)C     [Br-]    BrC(CC#N)(C)C             
3                [Br-]        BrC(C)(C)     [Br-]        BrC(C)(C)             
4                [Br-]          BrC(C)C     [Br-]          BrC(C)C             
...                ...              ...       ...              ...       ...   
1293      BrC(C(N)NO)N                -     [Br-]       C(C(N)NO)N             
1294   BrC(C(N)NO)(NO)             [F-]     [Br-]  C(=C(NO))(\N)NO         F   
1295   BrC(C(N)NO)(NO)                -     [Br-]  C(=C(NO))(\N)NO             
1296  BrC(C(N)NO)(NO)N             [F-]     [Br-]  FC(C(N)NO)(NO)N             
1297   BrC(C(N)NO)(N)C             [F-]     [Br-]  C(=C(N)C)(\N)NO         F   

          yield                        

In [253]:
# print the reaction
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

# Create a reaction SMARTS string
reaction_smarts = split_dataFrame.iloc[max_yield_row_index, 6]
print (reaction_smarts)
# Parse the reaction SMARTS string
reaction = AllChem.ReactionFromSmarts(reaction_smarts)

# Draw the reaction
reaction_image = Draw.ReactionToImage(reaction)

# Save or display the image
reaction_image.show()

BrC(CN)N.[F-]>>[Br-].C(=CN)N.F


In [162]:
from rdkit import Chem
from rdkit.Chem import Draw

# SMILES representation of the molecule
smiles = "FC(C(N)(NO))N"

# Create a molecule object from the SMILES string
mol = Chem.MolFromSmiles(smiles)

# Check if the molecule object is not None (i.e., if the SMILES string is valid)
if mol is not None:
    # Draw and display the molecule
    img = Draw.MolToImage(mol)
    img.show()
else:
    print("Invalid SMILES string")